In [1]:
# 导入相关库
# jinyb 2023-4-7
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
print('成功建立开发环境')

成功建立开发环境


In [2]:
# 借助folium库绘制中国地区疫情热力图
# 读取数据
df = pd.read_csv('DXYArea.csv')
print(df.head())

  provinceName cityName  province_confirmedCount  province_suspectedCount  \
0          浙江省       温州                     1075                        0   
1          浙江省       杭州                     1075                        0   
2          浙江省       宁波                     1075                        0   
3          浙江省       台州                     1075                        0   
4          浙江省       金华                     1075                        0   

   province_curedCount  province_deadCount  city_confirmedCount  \
0                  190                   0                  448   
1                  190                   0                  165   
2                  190                   0                  146   
3                  190                   0                  138   
4                  190                   0                   53   

   city_suspectedCount  city_curedCount  city_deadCount  \
0                    0               71               0   
1               

In [3]:
# 按照省分统计人数
df = df.groupby('provinceName').province_confirmedCount.agg('max').reset_index()
print(df.head())

  provinceName  province_confirmedCount
0          上海市                      293
1          云南省                      140
2       内蒙古自治区                       54
3          北京市                      326
4          吉林省                       78


In [4]:
# 读取各城市坐标数据，并给每一列命名，分别为postcode、city、lat、lon
df_city = pd.read_csv('china_coordinates.csv', names=['postcode', 'city', 'lat', 'lon'])
print(df_city.head())

   postcode city         lat        lon
0    110000  北京市  116.395645  39.929986
1    110101  东城区  113.612838  37.857865
2    110102  西城区  113.612838  37.857865
3    110105  朝阳区  116.521695  39.958953
4    110106  丰台区  116.258370  39.841938


In [5]:
# 合并城市坐标数据与疫情确诊数据
df_city['confirm'] = df_city['city'].map(df.set_index('provinceName')['province_confirmedCount'])
print(df_city.head())

   postcode city         lat        lon  confirm
0    110000  北京市  116.395645  39.929986    326.0
1    110101  东城区  113.612838  37.857865      NaN
2    110102  西城区  113.612838  37.857865      NaN
3    110105  朝阳区  116.521695  39.958953      NaN
4    110106  丰台区  116.258370  39.841938      NaN


In [6]:
# 去掉空值
df_city = df_city.dropna()
# 重新设置索引
df_city = df_city.reset_index(drop=True)
print(df_city.head())

   postcode    city         lat        lon  confirm
0    110000     北京市  116.395645  39.929986    326.0
1    120000     天津市  117.210813  39.143930     90.0
2    130000     河北省  115.661434  38.613840    206.0
3    140000     山西省  112.515496  37.866566    115.0
4    150000  内蒙古自治区  114.415868  43.468238     54.0


In [7]:
# folium
map = folium.Map(location = [39.929986, 116.395645], zoom_start = 4)

df_city['logconfirm'] = np.log(df_city['confirm'])
folium.Choropleth(
    name= '疫情热图',
    geo_data=open('100000_full.json',encoding = 'utf-8').read(),
    data = df_city.set_index(df_city.city),
    columns = ['city', 'logconfirm'],
    key_on = 'feature.properties.name',
    fill_color = 'Reds',
).add_to(map)

In [8]:
# 为地图添加标记，并显示经纬度信息
for i in range(len(df_city)):
    folium.Marker(
        location = [df_city.iloc[i]['lon'], df_city.iloc[i]['lat']],
        popup = df_city.iloc[i]['city'] + '：' + str(df_city.iloc[i]['confirm']),
        icon = folium.Icon(color = 'red', icon = 'info-sign')
    ).add_to(map)
map
# 保存地图
map.save('map.html')

In [9]:
# jinyb 2023-4-7